In [2]:
import os
import sys

sys.path.append(os.path.join(os.getcwd(), '..','..','..'))
from configs import spark_config as config
from utils import spark_utils as utils

# Specify the directory where your CSV files are located
directory = config.RAW_DATA_DIR 

# sparksession
spark = utils.create_spark_session("T20 Team Stats")

team_data = spark.read.csv(os.path.join(directory, 't20_team_stats.csv'), header=True, inferSchema=True)
team_data.show()

/home/ravikumar/miniconda3/envs/t20i/lib/python3.12/site-packages/airflow/configuration.py:859 FutureWarning: section/key [core/sql_alchemy_conn] has been deprecated, you should use[database/sql_alchemy_conn] instead. Please update your `conf.get*` call to use the new name

[2024-11-24T13:43:28.445+0530] {spark_utils.py:17} INFO - Creating Spark session.


your 131072x1 screen size is bogus. expect trouble
24/11/24 13:43:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/24 13:43:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


[2024-11-24T13:43:39.399+0530] {spark_utils.py:37} INFO - Spark session created successfully.


+------------+---+---+----+----+---+-----+-----+-----+----+---+---+-------+-----------+
|        Team|Mat|Won|Lost|Tied| NR|  W/L|  Ave|  RPO|Inns| HS| LS| Season|Unnamed: 13|
+------------+---+---+----+----+---+-----+-----+-----+----+---+---+-------+-----------+
|   Australia|  1|  1|   0|   0|  0|    -|42.80| 10.7|   1|214|  -|2004/05|       null|
| New Zealand|  1|  0|   1|   0|  0|0.000|17.00|  8.5|   1|170|170|2004/05|       null|
|   Australia|  1|  0|   1|   0|  0|0.000| 7.90| 5.44|   1| 79| 79|   2005|       null|
|     England|  1|  1|   0|   0|  0|    -|22.37| 8.95|   1|179|  -|   2005|       null|
|   Australia|  2|  1|   1|   0|  0|1.000|40.80| 10.2|   2|209|  -|2005/06|       null|
| New Zealand|  2|  1|   0|   1|  0|    -|20.00| 6.84|   2|134|  -|2005/06|       null|
|South Africa|  3|  1|   2|   0|  0|0.500|18.66| 7.72|   3|201|114|2005/06|       null|
| West Indies|  1|  0|   0|   1|  0|0.000|18.00|  6.3|   1|126|  -|2005/06|       null|
|     England|  2|  0|   2|   0|

In [3]:
from pyspark.sql.functions import col,when,round
team_data = team_data.withColumn("W/L", round(when(col("Lost")==0, col("Won")).otherwise(col("Won")/col("Lost")),2))
team_data = team_data.withColumn("AveRPW", when(col("Ave")=='-',0).otherwise(col("Ave")).cast("float")).drop("Ave")
team_data = team_data.withColumn("AveRPO", when(col("RPO")=='-',0).otherwise(col("RPO")).cast("float")).drop("RPO","LS")
team_data.show()

+------------+---+---+----+----+---+---+----+---+-------+-----------+------+------+
|        Team|Mat|Won|Lost|Tied| NR|W/L|Inns| HS| Season|Unnamed: 13|AveRPW|AveRPO|
+------------+---+---+----+----+---+---+----+---+-------+-----------+------+------+
|   Australia|  1|  1|   0|   0|  0|1.0|   1|214|2004/05|       null|  42.8|  10.7|
| New Zealand|  1|  0|   1|   0|  0|0.0|   1|170|2004/05|       null|  17.0|   8.5|
|   Australia|  1|  0|   1|   0|  0|0.0|   1| 79|   2005|       null|   7.9|  5.44|
|     England|  1|  1|   0|   0|  0|1.0|   1|179|   2005|       null| 22.37|  8.95|
|   Australia|  2|  1|   1|   0|  0|1.0|   2|209|2005/06|       null|  40.8|  10.2|
| New Zealand|  2|  1|   0|   1|  0|1.0|   2|134|2005/06|       null|  20.0|  6.84|
|South Africa|  3|  1|   2|   0|  0|0.5|   3|201|2005/06|       null| 18.66|  7.72|
| West Indies|  1|  0|   0|   1|  0|0.0|   1|126|2005/06|       null|  18.0|   6.3|
|     England|  2|  0|   2|   0|  0|0.0|   2|161|   2006|       null| 25.41|

In [4]:
# Cumulative calculations
from pyspark.sql import Window
from pyspark.sql.functions import col, sum as spark_sum, when, row_number, round

# Define the window specification for cumulative calculations
window_spec = Window.partitionBy("Team").orderBy("Season").rowsBetween(Window.unboundedPreceding, -1)

# Window for row number to identify the first row per player and country
row_num_window = Window.partitionBy("Team").orderBy("Season")

# perform cumulative calculations
team_data = team_data.withColumn("row_num", row_number().over(row_num_window)) \
    .withColumn("Cumulative Won",
                when(col("row_num") == 1, 0)
                .otherwise(spark_sum("Won").over(window_spec))) \
    .withColumn("Cumulative Lost",
                when(col("row_num") == 1, 0)  # Set 0 for the first row (before any match)
                .otherwise(spark_sum("Lost").over(window_spec))) \
    .withColumn("Cumulative Tied", 
                when(col("row_num") == 1, 0)  # Set 0 for the first row (before any match)
                .otherwise(spark_sum("Tied").over(window_spec))) \
    .withColumn("Cumulative NR", 
                when(col("row_num") == 1, 0)
                .otherwise(spark_sum("NR").over(window_spec))) \
    .withColumn("Cumulative W/L", 
                when(col("row_num") == 1, 0)
                .otherwise(
                    round(
                        when(spark_sum("Lost").over(window_spec) != 0, 
                             spark_sum(("Won")).over(window_spec) / spark_sum("Lost").over(window_spec))
                        .otherwise(0), 2)
                )
    ) \
    .withColumn("Cumulative AveRPW", 
                when(col("row_num") == 1, 0)
                .otherwise(
                    round(
                        when(spark_sum("Won").over(window_spec) != 0, 
                             spark_sum(col("AveRPW")*col("Mat")).over(window_spec) / spark_sum("Mat").over(window_spec))
                        .otherwise(0), 2)
                )
    ) \
    .withColumn("Cumulative AveRPO", 
                when(col("row_num") == 1, 0)
                .otherwise(
                    round(
                        when(spark_sum("Lost").over(window_spec) != 0, 
                             spark_sum(col("AveRPO")*col("Mat")).over(window_spec) / spark_sum("Mat").over(window_spec))
                        .otherwise(0), 2)
                )
    ) \
    .drop("row_num")  # Drop the temporary row number column

# Show the resulting DataFrame
team_data.show(10)

+-----------+---+---+----+----+---+----+----+---+-------+-----------+------+------+--------------+---------------+---------------+-------------+--------------+-----------------+-----------------+
|       Team|Mat|Won|Lost|Tied| NR| W/L|Inns| HS| Season|Unnamed: 13|AveRPW|AveRPO|Cumulative Won|Cumulative Lost|Cumulative Tied|Cumulative NR|Cumulative W/L|Cumulative AveRPW|Cumulative AveRPO|
+-----------+---+---+----+----+---+----+----+---+-------+-----------+------+------+--------------+---------------+---------------+-------------+--------------+-----------------+-----------------+
|Afghanistan|  6|  4|   2|   0|  0| 2.0|   6|147|2009/10|       null| 20.22|  6.89|             0|              0|              0|            0|           0.0|              0.0|              0.0|
|Afghanistan|  2|  0|   2|   0|  0| 0.0|   2|115|   2010|       null| 10.83|  5.41|             4|              2|              0|            0|           2.0|            20.22|             6.89|
|Afghanistan|  3|  2

In [7]:
team_data = team_data.select("Team", "Season","Cumulative Won", "Cumulative Lost", "Cumulative Tied", "Cumulative NR", "Cumulative W/L", "Cumulative AveRPW", "Cumulative AveRPO")
team_data.show()

+-----------+-------+--------------+---------------+---------------+-------------+--------------+-----------------+-----------------+
|       Team| Season|Cumulative Won|Cumulative Lost|Cumulative Tied|Cumulative NR|Cumulative W/L|Cumulative AveRPW|Cumulative AveRPO|
+-----------+-------+--------------+---------------+---------------+-------------+--------------+-----------------+-----------------+
|Afghanistan|2009/10|             0|              0|              0|            0|           0.0|              0.0|              0.0|
|Afghanistan|   2010|             4|              2|              0|            0|           2.0|            20.22|             6.89|
|Afghanistan|2011/12|             4|              4|              0|            0|           1.0|            17.87|             6.52|
|Afghanistan|2012/13|             6|              5|              0|            0|           1.2|            19.18|             6.92|
|Afghanistan|2013/14|             8|              7|          

In [8]:
team_data.write.csv(os.path.join(config.PROCESSED_DATA_DIR, 'team_stats.csv'), header=True, mode='overwrite')
spark.stop()